# Loading the Data

Loading the transformed data frame, previously saved on the Data Engineering stage

In [0]:
df = spark.read.load("/FileStore/tables/finalAIRBNB_noText")
df_2= spark.read.load("/FileStore/tables/finalAIRBNB_withText")

In [0]:
display(df)

id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,priceLog,clean_name,neighbourhood_groupidx,neighbourhood_groupcatVec,room_typeidx,room_typecatVec,vector,vector_ns,bigrams,textBOW,fts
2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1.0,45.0,2019-05-21,0.38,2.0,355.0,2.3521826,skylit midtown castle,0.0,"Map(vectorType -> sparse, length -> 67, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 76, indices -> List(0), values -> List(1.0))","List(skylit, midtown, castle)","List(skylit, midtown, castle)","List(skylit midtown, midtown castle)","Map(vectorType -> sparse, length -> 16264, indices -> List(), values -> List())","Map(vectorType -> sparse, length -> 146, indices -> List(0, 1, 2, 69, 145), values -> List(1.0, 45.0, 1.0, 1.0, 2.0))"
5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10.0,9.0,2018-11-19,0.10,1.0,0.0,1.90309,entire apt spacious studioloft by central park,0.0,"Map(vectorType -> sparse, length -> 67, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 76, indices -> List(0), values -> List(1.0))","List(entire, apt, spacious, studioloft, by, central, park)","List(entire, apt, spacious, studioloft, central, park)","List(entire apt, apt spacious, spacious studioloft, studioloft central, central park)","Map(vectorType -> sparse, length -> 16264, indices -> List(3, 244, 6361), values -> List(1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 146, indices -> List(0, 1, 2, 69, 145), values -> List(10.0, 9.0, 1.0, 1.0, 1.0))"
5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.975,Entire home/apt,200,3.0,74.0,2019-06-22,0.59,1.0,129.0,2.30103,large cozy 1 br apartment in midtown east,0.0,"Map(vectorType -> sparse, length -> 67, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 76, indices -> List(0), values -> List(1.0))","List(large, cozy, 1, br, apartment, in, midtown, east)","List(large, cozy, 1, br, apartment, midtown, east)","List(large cozy, cozy 1, 1 br, br apartment, apartment midtown, midtown east)","Map(vectorType -> sparse, length -> 16264, indices -> List(16, 76, 108, 218, 524, 742), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 146, indices -> List(0, 1, 2, 69, 145), values -> List(3.0, 74.0, 1.0, 1.0, 1.0))"
5238,Cute & Cozy Lower East Side 1 bdrm,7549,Ben,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1.0,160.0,2019-06-09,1.33,4.0,188.0,2.1760912,cute cozy lower east side 1 bdrm,0.0,"Map(vectorType -> sparse, length -> 67, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 76, indices -> List(0), values -> List(1.0))","List(cute, , cozy, lower, east, side, 1, bdrm)","List(cute, , cozy, lower, east, side, 1, bdrm)","List(cute , cozy, cozy lower, lower east, east side, side 1, 1 bdrm)","Map(vectorType -> sparse, length -> 16264, indices -> List(5, 27, 34, 88, 737, 744, 4619), values -> List(1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0))","Map(vectorType -> sparse, length -> 146, indices -> List(0, 1, 2, 69, 145), values -> List(1.0, 160.0, 1.0, 1.0, 4.0))"
5295,Beautiful 1br on Upper West Side,7702,Lena,Manhattan,Upper West Side,40.80316,-73.96545,Entire home/apt,135,5.0,53.0,2019-06-22,0.43,1.0,6.0,2.1303337,beautiful 1br on upper west side,0.0,"Map(vectorType -> sparse, length -> 67, indices -> List(0), values -> List(1.0))",0.0,"Map(vectorType -> sparse, length -> 76, indices -> List(0), values -> List(1.0))","List(beautiful, 1br, on, upper, west, side)","List(beautiful, 1br, upper, west, side)","List(beautiful 1br, 1br upper, upper west, west side)","Map(vectorType -> sparse, length -> 16264, indices -> List(17, 19, 636

# Splitting Train and Test Sets

In [0]:
(trainingData, testData) = df.select(['fts','priceLog']).randomSplit([0.8, 0.2],seed = 11)
(trainingData_2, testData_2) = df_2.select(['fts','priceLog']).randomSplit([0.8, 0.2],seed = 11)

# GBT Regressor Model

After some experimentation, maxDepth was defined as 7, since after this amount, there is not much of improvement on both R2 and RMSE scores although the increasing training time.

In [0]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(featuresCol="fts", labelCol='priceLog',maxDepth=5)
gbt_model = gbt.fit(trainingData)
pred = gbt_model.transform(testData)


Training now with the BOW feature

In [0]:
gbt_model_2 = gbt.fit(trainingData_2)
pred_2 = gbt_model_2.transform(testData_2)

# Model Evaluation

Performance without BOW feature

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="priceLog",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(pred))
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="priceLog",metricName="rmse")
print("RMSE on test data = %g" % lr_evaluator.evaluate(pred))

R Squared (R2) on test data = 0.554456
RMSE on test data = 0.180695

Performance with BOW feature

In [0]:
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="priceLog",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(pred_2))
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="priceLog",metricName="rmse")
print("RMSE on test data = %g" % lr_evaluator.evaluate(pred))

R Squared (R2) on test data = 0.55758
RMSE on test data = 0.180695

# Exporting Train and Test Sets

In [0]:
trainingData.write.save("/FileStore/tables/trainingData_noText",format='parquet')
testData.write.save("/FileStore/tables/testData_noText",format='parquet')
trainingData_2.write.save("/FileStore/tables/trainingData_2_noText",format='parquet')
testData_2.write.save("/FileStore/tables/testData_2_noText",format='parquet')

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-755679793723808> in <module> 
 ----> 1 trainingData . write . save ( "/FileStore/tables/trainingData_noText" , format = 'parquet' ) 
 2 testData . write . save ( "/FileStore/tables/testData_noText" , format = 'parquet' ) 
 3 trainingData_2 . write . save ( "/FileStore/tables/trainingData_2_noText" , format = 'parquet' ) 
 4 testData_2 . write . save ( "/FileStore/tables/testData_2_noText" , format = 'parquet' ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 828 self . _jwrite . save ( ) 
 829 else : 
 --> 830 self . _jwrite . save ( path ) 
 831 
 832 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : path dbfs:/FileStore/tables/trainingData_noText already exists.;